In [1]:
# import libraries
import os
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# images file path
train_root_path = "./dataset/train"
test_root_path = "./dataset/test"

In [3]:
# get train images and labels
train_names = os.listdir(train_root_path)
train_images = []
train_images_class_id = []
for index, name in enumerate(train_names):
    curdir = os.path.join(train_root_path, name)
    for path in os.listdir(curdir):
        img = cv2.imread(os.path.join(curdir, path))
        img_array = np.array(img)
        train_images.append(img_array)
        train_images_class_id.append(index)

In [4]:
# detect faces and filter train images
detector = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

filtered_train_images = []
filtered_train_images_id = []
for i, image in enumerate(train_images):
    # grayscaling
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # apply gaussian
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    # detect faces
    faces = detector.detectMultiScale(gray, scaleFactor=1.07, minNeighbors=15)

    # only valid if there's 1 face detected
    if(len(faces) == 1):
        (x,y,w,h) = faces[0]
        cropped = gray[y:y+h, x:x+w]
        filtered_train_images.append(cropped)
        filtered_train_images_id.append(train_images_class_id[i])    

In [5]:
# train face recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(filtered_train_images, np.array(filtered_train_images_id))

In [6]:
# get test images
test_images = []

for path in os.listdir(test_root_path):
    img = cv2.imread(os.path.join(test_root_path, path))
    img_array = np.array(img)
    test_images.append(img_array)

In [7]:
# detect faces and filter test images
filtered_test_images = []
test_image_rects = []

for images in test_images:
    # grayscaling
    gray = cv2.cvtColor(images, cv2.COLOR_BGR2GRAY)
    # apply gaussian
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    # detect faces
    faces = detector.detectMultiScale(gray, scaleFactor=1.07, minNeighbors=15)

    if(len(faces) == 1):
        (x,y,w,h) = faces[0]
        crop_img = gray[y:y+h, x:x+w]
        filtered_test_images.append(crop_img)
        test_image_rects.append(faces)

In [8]:
# predict results
predictions = []

for img in filtered_test_images:
    res, loss = face_recognizer.predict(img)
    loss = math.floor(loss * 100) / 100
    predictions.append((res, loss))

In [9]:
# show prediction results
drawn_test_images = []
size = 200
for img, rect, (prediction, loss) in zip(test_images, test_image_rects, predictions):
    (x, y, w, h) = (rect[0])
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 3)
    resized = cv2.resize(img, (size, size))
    text = train_names[prediction]
    lossText = str(loss)
    cv2.putText(resized, text, (10,20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)
    cv2.putText(resized, lossText, (100,20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)
    
    drawn_test_images.append(resized)

# combine all images
combined_image = cv2.hconcat(drawn_test_images)
cv2.imshow('img', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()